In [3]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import openai
import os


OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


response_schemas = [
    ResponseSchema(name="current_output", description="Output from the current agent that will be passed to the next agent. If there is no next agent, put None"),
    ResponseSchema(name="next_agent", description="The next agent to handle the request."),
    ResponseSchema(name="next_task", description="The task or action for the next agent."),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [4]:
format_instructions = output_parser.get_format_instructions()

In [5]:
prompt = PromptTemplate(
    template="answer the query or do the given task as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [8]:
model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser

response = chain.invoke({"question": "What is 1+1?"})
print(response["next_agent"])